In [179]:
# Workaround to use src modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [180]:
# Library imports
import imageio
import numpy as np
import matplotlib.pyplot as plt

from skimage import filters
from skimage.color import rgb2lab
from scipy.stats import mode

# Functions Imports
from src.utils import image_to_grayscale, image_minmax_norm
from src.quantization import image_bitshift
from src.segmentation import threshold_segmentation, otsu_segmentation
from src.visualizations import visualize_bitshift_images

#flood
from skimage.morphology import flood

#flood fill
from skimage.morphology import flood_fill

## Objetivo

Explorar técnicas de extração de paleta de cores para redução da quantidade de cores necessárias na imagem final para um N pré-selecionado.

## 1º Método - Utilizando Kmeans clustering

O primeiro método utiliza o algoritmo de aprendizado não supervisionado para gerar uma paleta de K cores que melhor representam o conjunto de cores da imagem. P/ isso utilizamos o algoritmo nas cores em diferentes espaços para verificar em qual deles as cores são melhores separáveis (RGB, LAB, HSV) além de aplicar métodos de correção gamma. 

In [181]:
from sklearn.cluster import KMeans
from scipy.ndimage import convolve
from scipy.ndimage import gaussian_filter

def palette_kmeans(image, N):
    # Pré-proccess: precisamos mudar a imagem de (N,N,3) p/ (N²,3)
    image_inline =  image.reshape(-1,3)
    
    # Training the model
    kmeans = KMeans(n_clusters=N, max_iter=300)
    kmeans.fit(image_inline)
    
    # Return Collor pallete (np.array)
    palette = kmeans.cluster_centers_
    
    # Reconstruct the image with the N colors
    image_labels = kmeans.predict(image_inline)
    image_labels = image_labels.reshape((image.shape[0], image.shape[1]))
    image_output = np.zeros(image.shape)
    
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            image_output[i,j,:] = palette[image_labels[i,j]]
    
    return palette, image_output, image_labels, kmeans

def labels_to_edges(labels):
    # Pre-proccess: add symetric pad to avoid corners bugs
    edge_map = np.zeros(labels.shape, dtype=int)
    labels   = np.pad(labels, 1, mode="symmetric")
    
    # Apply Laplacian filter to detect if any 8-neighboor is diff
    # not edge -> 0 (black)
    # edge     -> 1 (white)
    for i in range(1,labels.shape[0]-1):
        for j in range(1,labels.shape[1]-1):
            same_color = np.sum(labels[i-1:i+2,j-1:j+2] == labels[i,j]) != 9
            # print(i,j,same_color)
            # print(labels[i-1:i+2,j-1:j+2])
            edge_map[i-1, j-1] = same_color * 1
    
    return edge_map

In [182]:
def plot_pallete(palette, shape):
    plt.figure(figsize=(12,1))
    for i in range(palette.shape[0]):
        plt.subplot(shape[0],shape[1],i+1)
        plt.axis("off")
        plt.imshow(np.ones((3,3,3), dtype=int)*palette[i].astype(int))
        
def plot_imagegrid(images, shape, figsize=(40,40)):
    plt.figure(figsize=figsize)
    for i in range(len(images)):
        plt.subplot(shape[0],shape[1],i+1)
        plt.imshow(images[i].astype(int), cmap="gray")

In [183]:
def exploratory_pipeline(image_path, N=8, palette_shape=(2,8), save_path=None):
    
    # Obs: Em imagens png ignorar o alpha usando image[:,:,:3]
    image = imageio.imread(image_path).astype(np.uint8)
    
    # Pipeline 1 - No Image Pre-proccess 
    # - Quantization to pallete with N  colors
    # - Edge detection based on image labes
    palette, image_kmean, labels, model = palette_kmeans(image[:,:,:3], N) # Obs: pode levar até 1 min :P
    return labels
    edges = labels_to_edges(labels)
    
    plot_pallete(palette, palette_shape)
    plot_imagegrid([image, image_kmean, edges], shape=(1,3), figsize=(40,40))
    
    # Pipeline 2 - Image enhance before quantize
    image_smooth = np.zeros(image[:,:,:3].shape)
    for c in range(3):
        image_smooth[:,:,c] = gaussian_filter(image[:,:,c], sigma=2, mode='nearest')
    palette, image_kmean, labels, model = palette_kmeans(image_smooth[:,:,:3], N) # Obs: pode levar até 1 min :P
    edges = labels_to_edges(labels)
    
    plot_pallete(palette, palette_shape)
    plot_imagegrid([image, image_kmean, edges], shape=(1,3), figsize=(40,40))
    
    if save_path is not None:
        imageio.imwrite(save_path, image_kmean)
        
    return image_kmean

In [184]:
#a = imageio.imread('../images/exploratory/afremov2_02_recolorized.jpg').astype(np.uint8)[:,:,:3]
# a[:,:,0] = gaussian_filter(a[:,:,0], sigma=2, mode='nearest')
# a[:,:,1] = gaussian_filter(a[:,:,1], sigma=2, mode='nearest')
# a[:,:,2] = gaussian_filter(a[:,:,2], sigma=2, mode='nearest')
# plt.imshow(a)

# imageio.imread('../images/raw/anime_girl.png').astype(np.uint8)[:,:,:3].shape
# gaussian_filter(imageio.imread('../images/raw/anime_girl.png').astype(np.uint8)[:,:,:3], sigma=5, mode='nearest').shape
# plt.imshow(gaussian_filter(imageio.imread('../images/raw/anime_girl.png').astype(np.uint8)[:,:,3], sigma=5, mode='nearest'))

In [185]:
labels = exploratory_pipeline('../images/raw/painting_afremov.jpg', N=16, palette_shape=(1,16), save_path=None)
print(labels)
print(type(labels))
print(type(labels[0]))
print(labels[0])
print(type(labels[0][0]))
print(labels[0][0])

[[ 4  1  1 ...  3  3  3]
 [ 1  1  1 ...  3  3  3]
 [ 4  1  1 ...  3  3  3]
 ...
 [ 1  1  1 ...  3 10 10]
 [ 1  1  1 ...  3  3  3]
 [ 1  1  1 ... 10 10 10]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
[4 1 1 ... 3 3 3]
<class 'numpy.int32'>
4


In [186]:
def kill_the_bitch(aux, vet, i):
    x = vet[i][0]
    y = vet[i][1]
    
    new_color = 100
    # esquerda -> diminuindo o y
    while (new_color == 100 and y != 0):
        if aux[x][y-1] != i:
            #achou cor nova!!!
            new_color = aux[x][y-1]
        else:
            y -= 1
            
    # direita -> aumentando o y
    while (new_color == 100 and y < aux.shape[1]):
        if aux[x][y+1] != i:
            #achou cor nova!!!
            new_color = aux[x][y+1]
            #TODO: verificar se a nova cor tbm deve ser eliminada
        else:
            y += 1
    
    #atualizar matriz
    aux[aux == i] = new_color
    
    #atualizar area
    vet[new_color*(-1) - 1][2] += vet[i][2]
    vet[i][2] = 0
    
    return aux, vet
    

In [208]:
#flood fill
#flood_fill(image, seed_point, new_value)
#image: An n-dimensional array
#seed_point: The point in image used as the starting point for the flood fill
#new_value: New value to set the entire fill
#Return: An array with the same shape as image is returned
#with values in areas connected to and equal (or within tolerance of) the seed point replaced with new_value

def image_flood_fill(aux):
    
    vet = []
    # oq tem no vet?
    # (x, y) - inicio da regiao
    # area
    
    # identifica a regiao encontrada
    label = -1
    
    # encontra posição para chamar o flood fill
    for x in range(aux.shape[0]):
        for y in range(aux.shape[1]):
            if aux[x][y] >= 0:
                print ("label eh ", label)
                aux = flood_fill(aux, (x, y), label)

                #contar quantos tem na regiao
                area = np.sum(aux == label)
                print ("e a area eh ", area)

                #armazena informações sobre a regiao em vet
                list_info = [x, y, area]
                vet.append(list_info)

                #decrementa o label da regiao
                label -= 1

    print (aux)

    #apagar regiões
    threshold = 3
    
    for i in vet:
        if vet[i][2] < threshold:
            aux, vet = kill_the_bitch(aux, vet, i)
              
    #atualizar aux
    for i in vet:
        if vet[i][2] != 0:
            aux[aux == i*(-1)-1] = image[vet[i][0], vet[i][1]]
            
    return aux.astype(np.uint8)

In [210]:
print(labels.shape) 
b = image_flood_fill(labels)
plt.imshow(b)

(1064, 1454)
label eh  -1
label eh  -2
label eh  -3
label eh  -4
label eh  -5
label eh  -6
label eh  -7
label eh  -8
label eh  -9
label eh  -10
label eh  -11
label eh  -12
label eh  -13
label eh  -14
label eh  -15
label eh  -16
label eh  -17
label eh  -18
label eh  -19
label eh  -20
label eh  -21
label eh  -22
label eh  -23
label eh  -24
label eh  -25
label eh  -26
label eh  -27
label eh  -28
label eh  -29
label eh  -30
label eh  -31
label eh  -32
label eh  -33
label eh  -34
label eh  -35
label eh  -36
label eh  -37
label eh  -38
label eh  -39
label eh  -40
label eh  -41
label eh  -42
label eh  -43
label eh  -44
label eh  -45
label eh  -46
label eh  -47
label eh  -48
label eh  -49
label eh  -50
label eh  -51
label eh  -52
label eh  -53
label eh  -54
label eh  -55
label eh  -56
label eh  -57
label eh  -58
label eh  -59
label eh  -60
label eh  -61
label eh  -62
label eh  -63
label eh  -64
label eh  -65
label eh  -66
label eh  -67
label eh  -68
label eh  -69
label eh  -70
label eh  -71
la

label eh  -568
label eh  -569
label eh  -570
label eh  -571
label eh  -572
label eh  -573
label eh  -574
label eh  -575
label eh  -576
label eh  -577
label eh  -578
label eh  -579
label eh  -580
label eh  -581
label eh  -582
label eh  -583
label eh  -584
label eh  -585
label eh  -586
label eh  -587
label eh  -588
label eh  -589
label eh  -590
label eh  -591
label eh  -592
label eh  -593
label eh  -594
label eh  -595
label eh  -596
label eh  -597
label eh  -598
label eh  -599
label eh  -600
label eh  -601
label eh  -602
label eh  -603
label eh  -604
label eh  -605
label eh  -606
label eh  -607
label eh  -608
label eh  -609
label eh  -610
label eh  -611
label eh  -612
label eh  -613
label eh  -614
label eh  -615
label eh  -616
label eh  -617
label eh  -618
label eh  -619
label eh  -620
label eh  -621
label eh  -622
label eh  -623
label eh  -624
label eh  -625
label eh  -626
label eh  -627
label eh  -628
label eh  -629
label eh  -630
label eh  -631
label eh  -632
label eh  -633
label eh  

label eh  -1122
label eh  -1123
label eh  -1124
label eh  -1125
label eh  -1126
label eh  -1127
label eh  -1128
label eh  -1129
label eh  -1130
label eh  -1131
label eh  -1132
label eh  -1133
label eh  -1134
label eh  -1135
label eh  -1136
label eh  -1137
label eh  -1138
label eh  -1139
label eh  -1140
label eh  -1141
label eh  -1142
label eh  -1143
label eh  -1144
label eh  -1145
label eh  -1146
label eh  -1147
label eh  -1148
label eh  -1149
label eh  -1150
label eh  -1151
label eh  -1152
label eh  -1153
label eh  -1154
label eh  -1155
label eh  -1156
label eh  -1157
label eh  -1158
label eh  -1159
label eh  -1160
label eh  -1161
label eh  -1162
label eh  -1163
label eh  -1164
label eh  -1165
label eh  -1166
label eh  -1167
label eh  -1168
label eh  -1169
label eh  -1170
label eh  -1171
label eh  -1172
label eh  -1173
label eh  -1174
label eh  -1175
label eh  -1176
label eh  -1177
label eh  -1178
label eh  -1179
label eh  -1180
label eh  -1181
label eh  -1182
label eh  -1183
label eh

label eh  -1641
label eh  -1642
label eh  -1643
label eh  -1644
label eh  -1645
label eh  -1646
label eh  -1647
label eh  -1648
label eh  -1649
label eh  -1650
label eh  -1651
label eh  -1652
label eh  -1653
label eh  -1654
label eh  -1655
label eh  -1656
label eh  -1657
label eh  -1658
label eh  -1659
label eh  -1660
label eh  -1661
label eh  -1662
label eh  -1663
label eh  -1664
label eh  -1665
label eh  -1666
label eh  -1667
label eh  -1668
label eh  -1669
label eh  -1670
label eh  -1671
label eh  -1672
label eh  -1673
label eh  -1674
label eh  -1675
label eh  -1676
label eh  -1677
label eh  -1678
label eh  -1679
label eh  -1680
label eh  -1681
label eh  -1682
label eh  -1683
label eh  -1684
label eh  -1685
label eh  -1686
label eh  -1687
label eh  -1688
label eh  -1689
label eh  -1690
label eh  -1691
label eh  -1692
label eh  -1693
label eh  -1694
label eh  -1695
label eh  -1696
label eh  -1697
label eh  -1698
label eh  -1699
label eh  -1700
label eh  -1701
label eh  -1702
label eh

label eh  -2169
label eh  -2170
label eh  -2171
label eh  -2172
label eh  -2173
label eh  -2174
label eh  -2175
label eh  -2176
label eh  -2177
label eh  -2178
label eh  -2179
label eh  -2180
label eh  -2181
label eh  -2182
label eh  -2183
label eh  -2184
label eh  -2185
label eh  -2186
label eh  -2187
label eh  -2188
label eh  -2189
label eh  -2190
label eh  -2191
label eh  -2192
label eh  -2193
label eh  -2194
label eh  -2195
label eh  -2196
label eh  -2197
label eh  -2198
label eh  -2199
label eh  -2200
label eh  -2201
label eh  -2202
label eh  -2203
label eh  -2204
label eh  -2205
label eh  -2206
label eh  -2207
label eh  -2208
label eh  -2209
label eh  -2210
label eh  -2211
label eh  -2212
label eh  -2213
label eh  -2214
label eh  -2215
label eh  -2216
label eh  -2217
label eh  -2218
label eh  -2219
label eh  -2220
label eh  -2221
label eh  -2222
label eh  -2223
label eh  -2224
label eh  -2225
label eh  -2226
label eh  -2227
label eh  -2228
label eh  -2229
label eh  -2230
label eh

label eh  -2682
label eh  -2683
label eh  -2684
label eh  -2685
label eh  -2686
label eh  -2687
label eh  -2688
label eh  -2689
label eh  -2690
label eh  -2691
label eh  -2692
label eh  -2693
label eh  -2694
label eh  -2695
label eh  -2696
label eh  -2697
label eh  -2698
label eh  -2699
label eh  -2700
label eh  -2701
label eh  -2702
label eh  -2703
label eh  -2704
label eh  -2705
label eh  -2706
label eh  -2707
label eh  -2708
label eh  -2709
label eh  -2710
label eh  -2711
label eh  -2712
label eh  -2713
label eh  -2714
label eh  -2715
label eh  -2716
label eh  -2717
label eh  -2718
label eh  -2719
label eh  -2720
label eh  -2721
label eh  -2722
label eh  -2723
label eh  -2724
label eh  -2725
label eh  -2726
label eh  -2727
label eh  -2728
label eh  -2729
label eh  -2730
label eh  -2731
label eh  -2732
label eh  -2733
label eh  -2734
label eh  -2735
label eh  -2736
label eh  -2737
label eh  -2738
label eh  -2739
label eh  -2740
label eh  -2741
label eh  -2742
label eh  -2743
label eh

KeyboardInterrupt: 